In [1]:
from glob import glob

import email
import smtplib
import imaplib
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
def str2int(message):
    exp = 1
    number = 0
    for i in range(len(message)):
        number += ord(message[i]) * exp
        exp *= 256
    
    return number

def int2str(number):
    message = ''
    while number != 0:
        temp = number % 256
        message += chr(temp)
        number -= temp
        number = number // 256
    
    return message

In [3]:
def encrypt(message, public_key):
    n, e = public_key
    m = str2int(message)
    m = pow(m, e, n)
    encrypted = int2str(m)
    
    return encrypted

def decrypt(encrypted, private_key):
    n, d = private_key
    m = str2int(encrypted)
    m = pow(m, d, n)
    message = int2str(m)
    
    return message

In [4]:
def read_email(username, password, private_key, tag = '[Encrypted]', qtd = 1):
    printed = 0
    crip = False
    server = 'imap.gmail.com'
    mail = imaplib.IMAP4_SSL(server)
    mail.login(username, password)
    mail.select('inbox')
    data = mail.search(None, 'ALL')
    mail_ids = data[1]
    id_list = mail_ids[0].split()   
    first_email_id = int(id_list[0])
    latest_email_id = int(id_list[-1])
    for i in range(latest_email_id, first_email_id, -1):
        if printed == qtd:
            break

        data = mail.fetch(str(i), '(RFC822)')
        for response_part in data:
            arr = response_part[0]
            if isinstance(arr, tuple):
                msg = email.message_from_string(str(arr[1], 'utf-8'))
                email_subject = msg['subject']
                email_from = msg['from']
                if tag not in email_subject:
                    break
                else:
                    printed += 1
                    crip = True

                print('From: ' + email_from)
                print('Subject: ' + email_subject)
                print()
                for part in msg.walk():
                    content_type = part.get_content_type()
                    content_disposition = str(part.get('Content-Disposition'))
                    try:
                        body = part.get_payload(decode = True).decode()
                    except:
                        pass

                    if content_type == 'text/plain':
                        body = decrypt(body, private_key)
                        print(body)
        
        if printed != qtd and crip:
            print()
            crip = False

In [5]:
if 'public.csv' not in glob('*.csv'):
    public_key, private_key = generate_keys(p, q)
    with open('public.csv', 'w') as file:
        file.write(str(public_key[0]) + '\n')
        file.write(str(public_key[1]) + '\n')
        
    with open('private.csv', 'w') as file:
        file.write(str(private_key[0]) + '\n')
        file.write(str(private_key[1]) + '\n')   
else:
    file = open('public.csv')
    public_key = file.readlines()
    file.close()
    public_key = tuple([int(i) for i in public_key])
    
    file = open('private.csv')
    private_key = file.readlines()
    file.close()
    private_key = tuple([int(i) for i in private_key])

In [6]:
username = input('Username: ')
password = getpass(prompt = 'Password: ')

Username: igorpmichels@gmail.com
Password: ········


In [7]:
read_email(username, password, private_key)

From: igorpmichels@gmail.com
Subject: [Encrypted] Eclass fora do ar?

O eclass ta fora do ar tbm? sei q zap, face e insta tão fora, mas tbm não consigo entrar no eclass. Como q vou estudar algebra? como q vou fazer a lista pra amanha?
